<a href="https://colab.research.google.com/github/lessw2020/training-detr/blob/master/training_detr_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Example of how to setup and train your own custom dataset using DETR
# In this notebook, we'll do fine tuning of the existing head
# For larger datasets (say > 10K images) training from scratch may be preferred and we'll do a different notebook for that



In [0]:
# Credit to @alcinos and @fmassa for their insights, help and for making DETR :) and @mlk1337 and @raviv for additional input 
# thread on custom training with DETR is here:  
# https://github.com/facebookresearch/detr/issues/9

# Setup PyTorch environment and DETR

In [1]:
!pip install torch torchvision


In [2]:
import torch; 
torch_version = torch.__version__
assert float(torch_version[:3]) >= 1.5   # make sure 1.5 or higher
torch_version


'1.5.0+cu101'

In [3]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'   #for coco apis


  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-dzj2pnvd
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-dzj2pnvd
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=267008 sha256=7e073082ec8a7cf991c22cdecb7cf84aa6f59d42fe0f915d1685bf673a9c861a
  Stored in directory: /tmp/pip-ephem-wheel-cache-38g2aufi/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0.0
    Uninstalling pycocotools-2.0.0:
      Successfully uninstalled pycocotools-2.0.0


In [4]:
!git clone https://github.com/facebookresearch/detr.git  #install detr 


Cloning into 'detr'...
remote: Enumerating objects: 115, done.
remote: Total 115 (delta 0), reused 0 (delta 0), pack-reused 115
Receiving objects: 100% (115/115), 7.27 MiB | 27.57 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [6]:
from pathlib import Path as p; p.cwd()  # make sure we know where we are

PosixPath('/content')

In [5]:
!ls 

detr  sample_data


In [7]:
%cd /content/detr  
# move default dir to detr to begin...

/content/detr


In [8]:
!ls   # make sure everything looks good

datasets    hubconf.py	models		  run_with_submitit.py	util
Dockerfile  LICENSE	README.md	  test_all.py
engine.py   main.py	requirements.txt  tox.ini


# Create a new model with blank fine tuning section for your dataset


In [0]:
from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
#from torchvision.models import resnet50
import torchvision.transforms as T

In [10]:
# download the latest pre-trained weights for detr with resnet50 backbone:
# detr resnet 50 backbone and head
!wget https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth

# or use resnet 101...we'll use res50 for this colab:
# !wget https://dl.fbaipublicfiles.com/detr/detr-r101-2c7b67e5.pth

# Additional info - skip if you like:

# 1 - over time, you want to check if these weight links above become outdated (i.e. newer weights are available) 
# weight listings are located at:  https://github.com/facebookresearch/detr#model-zoo
  
# 2 - Further note - there are DC5 (dc = dilated convolutions) models for both resnet50 and resnet101.  
# Both are a bit more accurate than their regular counterpart, but increase memory consumption 2x 
# and are hard to train as a result (CUDA out of memory errors)...we will use plain resnet50 here.

--2020-06-14 21:28:23--  https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166618694 (159M) [application/octet-stream]
Saving to: ‘detr-r50-e632da11.pth’

detr-r50-e632da11.p 100%[===================>] 158.90M  22.3MB/s    in 7.7s    

2020-06-14 21:28:31 (20.6 MB/s) - ‘detr-r50-e632da11.pth’ saved [166618694/166618694]



In [0]:
# we need to load the weights, clip out the two layers we want to train, and then save it back out:
checkpoint = torch.load("detr-r50-e632da11.pth", map_location='cpu')
del checkpoint["model"]["class_embed.weight"]
del checkpoint["model"]["class_embed.bias"]
torch.save(checkpoint,"detr-r50_ready_to_train.pth")


In [0]:
# build a model for training and adjust num_classes for your dataset

In [0]:
import argparse
import datetime
import json
import random
import time
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader, DistributedSampler

import datasets
import util.misc as utils
from datasets import build_dataset, get_coco_api_from_dataset
from engine import evaluate, train_one_epoch
from models import build_model



In [0]:
# pull the args from main.py (the main detr training script) so we can use here in colab to make our model
from main import get_args_parser  
args_parser = get_args_parser()
args = args_parser.parse_args("")


In [14]:
args

Namespace(aux_loss=True, backbone='resnet50', batch_size=2, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path=None, coco_path=None, dataset_file='coco', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=300, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=0.0001, lr_backbone=1e-05, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_queries=100, num_workers=2, output_dir='', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weight_decay=0.0001, world_size=1)

In [0]:
# Customize your dataset  - must be COCO format.
# I created my own wrapper to remap class ids, show how to do that?  is that best way...


In [0]:
# adjust num_classes - via command line or adjust code (PR?)

In [0]:
# training - in notebook as exposed or run as shell?

In [0]:
# visuals - check plots for progress
